In [3]:
import pandas as pd
import numpy as np
import string
import pickle
from nltk.corpus import stopwords

In [4]:
master = pd.read_csv('dfMaster.csv')
train = pd.read_csv('trainMaster.csv')
final = pd.read_csv('dfTestDataLabelFinal.csv')

In [9]:
train.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,toxic_level,clean,length,word_count,polarity,subjectivity
0,0,0000997932d777bf,Explanation Why the edits made under my userna...,0,0,0,0,0,0,0,1,264,43,0.136364,0.454545
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0,1,112,18,0.287500,0.550000
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0,1,233,42,0.160000,0.406667
3,3,0001b41b1c6bb37e,""" More I can't make any real suggestions on im...",0,0,0,0,0,0,0,1,622,116,0.200000,0.301042
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0,1,67,13,0.000000,0.000000


In [11]:
final.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,toxic_level,clean,length,word_count,polarity,subjectivity
0,0001ea8717f6de06,Explanation Why the edits made under my userna...,0,0,0,0,0,0,0,1,96,16,0.208000,0.702000
1,000247e83dcc1211,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0,1,32,6,-1.000000,1.000000
2,0002f87b16116a7f,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0,1,448,69,-0.001562,0.576563
3,0003e1cccfd5a40a,""" More I can't make any real suggestions on im...",0,0,0,0,0,0,0,1,501,85,0.151948,0.406494
4,00059ace3e3e9a53,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0,1,334,57,0.286580,0.631457


In [5]:
def preprocess(ReviewText):
    ReviewText = ReviewText.str.replace("(<br/>)", "")
    ReviewText = ReviewText.str.replace('(<a).*(>).*(</a>)', '')
    ReviewText = ReviewText.str.replace('(&amp)', '')
    ReviewText = ReviewText.str.replace('(&gt)', '')
    ReviewText = ReviewText.str.replace('(&lt)', '')
    ReviewText = ReviewText.str.replace('(\xa0)', ' ')
    ReviewText = ReviewText.str.replace('(\n)', ' ') 
    ReviewText = ReviewText.str.replace('==', ' ') 
    return ReviewText

master['comment_text'] = preprocess(master['comment_text'])
train['comment_text'] = preprocess(train['comment_text'])
final['comment_text'] = preprocess(master['comment_text'])

# Target

In [4]:
x = train['comment_text']

categories = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

# Model

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, matthews_corrcoef, f1_score, log_loss

# Analyzer for Pipeline

In [6]:
stop = []
for words in stopwords.words('english'):
    s = [char for char in words if char not in string.punctuation]
    stop.append(''.join(s))
stop.extend(['may','also','across','among','beside','however','yet','within'])

def process_nocasepunc(text):
    nocasepunc = [char for char in text if char not in string.punctuation]
    nocasepunc = ''.join(nocasepunc)
    return [word.lower() for word in nocasepunc.split() if word.lower() not in stop]

# Pipeline

In [8]:
LogReg_pipeline_toxic = Pipeline([
            ('vectorizer', TfidfVectorizer(analyzer='word', ngram_range=(1,3), max_features=30000)),
            ('classifier', LogisticRegression()),
        ])
LogReg_pipeline_severe_toxic = Pipeline([
            ('vectorizer', TfidfVectorizer(analyzer='word', ngram_range=(1,3), max_features=30000)),
            ('classifier', LogisticRegression()),
        ])
LogReg_pipeline_obscene = Pipeline([
            ('vectorizer', TfidfVectorizer(analyzer='word', ngram_range=(1,3), max_features=30000)),
            ('classifier', LogisticRegression()),
        ])
LogReg_pipeline_threat = Pipeline([
            ('vectorizer', TfidfVectorizer(analyzer='word', ngram_range=(1,3), max_features=30000)),
            ('classifier', LogisticRegression()),
        ])
LogReg_pipeline_insult = Pipeline([
            ('vectorizer', TfidfVectorizer(analyzer='word', ngram_range=(1,3), max_features=30000)),
            ('classifier', LogisticRegression()),
        ])
LogReg_pipeline_identity_hate = Pipeline([
            ('vectorizer', TfidfVectorizer(analyzer='word', ngram_range=(1,3), max_features=30000)),
            ('classifier', LogisticRegression()),
        ])

# # def pipeline_model_training(category, dataTrain, targetTrain):
# #     # Training logistic regression model on train data
# #     return LogReg_pipeline.fit(dataTrain, targetTrain)

# # def pipeline_predict(category, dictPred, dictPredProba, dictHasil, dataTest, targetTest):
# #     # calculating test accuracy
# #     prediction = LogReg_pipeline.predict(dataTest)
# #     dictPredProba['{}'.format(category)] = LogReg_pipeline.predict_proba(dataTest)
# #     dictPred[category] = prediction 
# #     dictHasil['{}_accuracy'.format(category)] = accuracy_score(targetTest, prediction)
# #     dictHasil['{}_f1score'.format(category)] = f1_score(targetTest, prediction, average = 'macro')
# #     dictHasil['{}_logloss'.format(category)] = log_loss(targetTest, dictPredProba[category])
# #     dictHasil['{}_rocauc'.format(category)] = roc_auc_score(targetTest, dictPredProba[category][:,1])
# #     dictHasil['{}_matthews_corrcoef'.format(category)] = matthews_corrcoef(targetTest, prediction)
    

In [10]:
LogReg_pipeline_clean = Pipeline([
            ('vectorizer', TfidfVectorizer(analyzer='word', ngram_range=(1,3), max_features=30000)),
            ('classifier', LogisticRegression()),
        ])
LogReg_pipeline_clean.fit(train['comment_text'],train['clean'])

C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=30000, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [12]:
predictClean = LogReg_pipeline_clean.predict(final['comment_text'])

In [22]:
final[final['clean']==0]['comment_text'].iloc[31]

"I'm pretty sure the incident has already been included. And I know how to handle my own at riots, too. 8)"

In [13]:
print(classification_report(final['clean'], predictClean))

              precision    recall  f1-score   support

           0       0.10      0.07      0.08      6243
           1       0.90      0.93      0.92     57735

   micro avg       0.85      0.85      0.85     63978
   macro avg       0.50      0.50      0.50     63978
weighted avg       0.82      0.85      0.83     63978



In [8]:
# LogReg_pipeline_toxic.fit(master['comment_text'],master['toxic'])
# LogReg_pipeline_severe_toxic.fit(master['comment_text'],master['severe_toxic'])
# LogReg_pipeline_obscene.fit(master['comment_text'],master['obscene'])
# LogReg_pipeline_threat.fit(master['comment_text'],master['threat'])
# LogReg_pipeline_insult.fit(master['comment_text'],master['insult'])
# LogReg_pipeline_identity_hate.fit(master['comment_text'],master['identity_hate'])

In [9]:
import pickle

In [10]:
# pickle.dump(LogReg_pipeline_toxic, open('LogReg_pipeline_toxic.sav','wb'))
# pickle.dump(LogReg_pipeline_severe_toxic, open('LogReg_pipeline_severe_toxic.sav','wb'))
# pickle.dump(LogReg_pipeline_obscene, open('LogReg_pipeline_obscene.sav','wb'))
# pickle.dump(LogReg_pipeline_threat, open('LogReg_pipeline_threat.sav','wb'))
# pickle.dump(LogReg_pipeline_insult, open('LogReg_pipeline_insult.sav','wb'))
# pickle.dump(LogReg_pipeline_identity_hate, open('LogReg_pipeline_identity_hate.sav','wb'))

In [11]:
LogReg_pipeline_toxic = pickle.load(open('LogReg_pipeline_toxic.sav','rb'))
LogReg_pipeline_severe_toxic = pickle.load(open('LogReg_pipeline_severe_toxic.sav','rb'))
LogReg_pipeline_obscene = pickle.load(open('LogReg_pipeline_obscene.sav','rb'))
LogReg_pipeline_threat = pickle.load(open('LogReg_pipeline_threat.sav','rb'))
LogReg_pipeline_insult = pickle.load(open('LogReg_pipeline_insult.sav','rb'))
LogReg_pipeline_identity_hate = pickle.load(open('LogReg_pipeline_identity_hate.sav','rb'))

In [12]:
def predict_text(text):
    hasil =[]
    proba = []
    proba.append(LogReg_pipeline_toxic.predict_proba([text])[0][1])
    proba.append(LogReg_pipeline_severe_toxic.predict_proba([text])[0][1])
    proba.append(LogReg_pipeline_obscene.predict_proba([text])[0][1])
    proba.append(LogReg_pipeline_threat.predict_proba([text])[0][1])
    proba.append(LogReg_pipeline_insult.predict_proba([text])[0][1])
    proba.append(LogReg_pipeline_identity_hate.predict_proba([text])[0][1])
    if LogReg_pipeline_toxic.predict([text])[0]==1:
        hasil.append('toxic')
    if LogReg_pipeline_severe_toxic.predict([text])[0]==1:
        hasil.append('severe toxic')
    if LogReg_pipeline_obscene.predict([text])[0]==1:
        hasil.append('obscene')
    if LogReg_pipeline_threat.predict([text])[0]==1:
        hasil.append('threat')
    if LogReg_pipeline_insult.predict([text])[0]==1:
        hasil.append('insult')
    if LogReg_pipeline_identity_hate.predict([text])[0]==1:
        hasil.append('identity hate')
        
    if(len(hasil)==0):
        textpred = 'This text is clean ^^'
    elif(len(hasil)==1):
        textpred = 'In conclusion, we predict that this text is a TOXIC text that contains {} content'.format(hasil[0])
    elif(len(hasil)==2):
        textpred = 'In conclusion, we predict that this text is a TOXIC text that contains {} and {} contents'.format(hasil[0],hasil[1])
    else:
        textpred = 'In conclusion, we predict that this text is a TOXIC text that contains {}, and {} contents'.format(', '.join(hasil[:-1]), hasil[-1])
    
    print('This text have probabilities of being toxic({}%), severe toxic({}%), obscene({}%), threat({}%), insult({}%), and identity hate({}%)'.format(round(proba[0]*100,2),round(proba[1]*100,2),round(proba[2]*100,2),round(proba[3]*100,2),round(proba[4]*100,2),round(proba[5]*100,2)))
    print(textpred)

In [13]:
text = 'you are asshole'
predict_text(text)

This text have probabilities of being toxic(99.98%), severe toxic(37.65%), obscene(99.96%), threat(0.69%), insult(99.94%), and identity hate(3.96%)
In conclusion, we predict that this text is a TOXIC text that contains toxic, obscene, and insult contents
